In [ ]:
!pip install ktrain

     |████████████████████████████████| 25.3 MB 84.5 MB/s 
     |████████████████████████████████| 6.8 MB 52.0 MB/s 
     |████████████████████████████████| 981 kB 48.4 MB/s 
     |████████████████████████████████| 263 kB 77.1 MB/s 
     |████████████████████████████████| 2.8 MB 55.0 MB/s 
     |████████████████████████████████| 1.2 MB 21.9 MB/s 
     |████████████████████████████████| 468 kB 69.3 MB/s 
     |████████████████████████████████| 3.3 MB 58.9 MB/s 
     |████████████████████████████████| 61 kB 595 kB/s 
     |████████████████████████████████| 596 kB 73.9 MB/s 
     |████████████████████████████████| 895 kB 83.9 MB/s 
  Created wheel for ktrain: filename=ktrain-0.28.3-py3-none-any.whl size=25292659 sha256=57e0ccefb9ec5c74e064ef5daf309f7789ef595aa63b8b3c7977647c125bd1fe
  Stored in directory: /root/.cache/pip/wheels/6a/7e/c3/f46cdfc2b81c54424923b1405d7e670c35cacc11ada9a47b1c
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9929 sha256=b3a2218daf4808c

In [ ]:
# Import the packages
#from rake_nltk import Rake
import operator
import pandas as pd
import numpy as np
import spacy
#import nltk
import re
from textblob import TextBlob
#import gensim
#from gensim import corpora, models
#from gensim.models.ldamulticore import LdaMulticore
#from nltk.corpus import stopwords
#from nltk.stem.wordnet import WordNetLemmatizer
import string
import multiprocessing
import warnings
#from gensim.utils import simple_preprocess
#from gensim.models import CoherenceModel
import os
import ktrain
from ktrain import text
import io 
from io import StringIO


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# the base Google Drive directory
root_dir = "/content/drive/My Drive/"
# choose where you want your project files to be saved
project_folder = "Colab Notebooks/NLP/yelp/"
data_folder = "Colab Notebooks/NLP/yelp/"
dataPath = root_dir+data_folder
modelPath = root_dir+model_folder
print("dataPath: " , dataPath)
print("modelPath: " , modelPath)

dataPath:  /content/drive/My Drive/Colab Notebooks/NLP/yelp/


In [ ]:
from google.colab import files
#uploaded = files.upload()

In [ ]:
yelp = pd.read_csv(dataPath + 'yelp.csv')
# Dataset is now stored in a Pandas Dataframe

In [ ]:
yelp.head()

,Unnamed: 0,text,stars,sentiment
0,2,I called WVM on the recommendation of a couple...,5,Positive
1,3,I've stayed at many Marriott and Renaissance M...,2,Negative
2,5,"This place used to be a cool, chill place. Now...",1,Negative
3,6,"The setting is perfectly adequate, and the foo...",2,Negative
4,7,Probably one of the better breakfast sandwiche...,5,Positive


In [ ]:
yelp.drop(['Unnamed: 0'], axis = 1, inplace = True)
yelp.shape

(314292, 3)

In [ ]:
yelp.head()

,text,stars,sentiment
0,I called WVM on the recommendation of a couple...,5,Positive
1,I've stayed at many Marriott and Renaissance M...,2,Negative
2,"This place used to be a cool, chill place. Now...",1,Negative
3,"The setting is perfectly adequate, and the foo...",2,Negative
4,Probably one of the better breakfast sandwiche...,5,Positive


#### Prepare the data

In [ ]:
sentiment = {1: "Negative", 2:"Negative", 4:"Positive" , 5: "Positive"}
yelp['sentiment'] = yelp['stars'].map(sentiment)

In [ ]:
yelp_sentiment = yelp[['text', 'sentiment']].rename(columns={'text':'data', 'sentiment':'target'})

In [ ]:
yelp_sentiment.head(5)

,data,target
0,I called WVM on the recommendation of a couple...,Positive
1,I've stayed at many Marriott and Renaissance M...,Negative
2,"This place used to be a cool, chill place. Now...",Negative
3,"The setting is perfectly adequate, and the foo...",Negative
4,Probably one of the better breakfast sandwiche...,Positive


In [ ]:
yelp_sentiment.groupby(['target']).count()

yelp_sentiment

,data,target
0,I called WVM on the recommendation of a couple...,Positive
1,I've stayed at many Marriott and Renaissance M...,Negative
2,"This place used to be a cool, chill place. Now...",Negative
3,"The setting is perfectly adequate, and the foo...",Negative
4,Probably one of the better breakfast sandwiche...,Positive
...,...,...
314287,It was like dining in Bavaria! Fun atmosphere ...,Positive
314288,Stayed here last month. Booked a room for 2 n...,Negative
314289,Aydrien Rayda Rocks!!\n\nFinding a hair stylis...,Positive
314290,Lunch buffet was fresh and delicious. Attenti...,Positive


In [ ]:
pd.set_option('display.max_rows', 500) # Set Max Number of Rows
pd.set_option('display.max_columns', 50) # Set Max Number of Columns
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000) # Set Max Width of Cell
pd.set_option('display.max_info_columns', 200) # Set Max Number of Columns Shown in info()
pd.set_option('display.precision', 6)#Set Display Precision of float values (0.123456)

In [ ]:
yelp_sentiment.data.str.split().apply(len)


0         114
1          72
2          57
3          34
4          82
         ... 
314287     99
314288    157
314289     67
314290     30
314291     63
Name: data, Length: 314292, dtype: int64

In [ ]:
yelp_sentiment.data[0:1]

0    I called WVM on the recommendation of a couple of friends who had used them in the past and thought they did a nice job. I'm a fan now, too.\n\nEvan and Cody showed up right on time for my move this past weekend. They were friendly and energetic, working quickly but carefully to get all my things moved out of the old place and into the new one in less than 2.5 hours. All of my (heavy) furniture arrived in perfect condition, and they took extra care not to scratch the wood floors in the process.\n\nI hope not to move again anytime soon, but next time I do, I'll be calling WVM.
Name: data, dtype: object

In [ ]:
df_yelp = yelp_sentiment.sample(frac=0.20)

In [ ]:
X_train = df_yelp.data.tolist()
Y_train = df_yelp.target.tolist()

#### STEP 1:  Load and Preprocess the Dataset

Preprocess the data using the texts_from_array function (since the data resides in an array). If your documents are stored in folders or a CSV file you can use the texts_from_folder or texts_from_csv functions, respectively.

In [ ]:
#maxFeatures = 20000 #num of words to consider in vocabulary
maxLen = 256 #each document can be of most <maxLen> words. 0 is used as padding ID.
nGramRange = 1 #size of multi-word phrases to consider
preprocessMode='bert' #Either 'standard' (normal tokenization) or 'bert' tokenization and preprocessing for use with BERT text classification model.
sampleSize = 0.3 #Proportion of training to use for validation

(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_df(train_df = df_yelp, 
                                                                    text_column  = 'data', 
                                                                    label_columns = ['target'],
                                                                       val_pct=sampleSize,
                                                                       preprocess_mode=preprocessMode, #text must be preprocessed in a specific way for use with BERT
                                                                       maxlen=maxLen)

['Negative', 'Positive']
        Negative  Positive
35663        0.0       1.0
274214       0.0       1.0
311997       0.0       1.0
179488       0.0       1.0
136183       0.0       1.0
['Negative', 'Positive']
        Negative  Positive
147557       1.0       0.0
154144       1.0       0.0
306251       1.0       0.0
50300        0.0       1.0
181917       0.0       1.0
preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


#### STEP 2:  Load a pretrained BERT model and wrap it in a `ktrain.Learner` object

In [ ]:
model = text.text_classifier('bert', (x_train, y_train), preproc=preproc)

Is Multi-Label? False
maxlen is 256
done.


In [ ]:
batchSize = 8 ### Check best size

learner = ktrain.get_learner(model, 
                             train_data=(x_train, y_train), 
                             val_data=(x_test, y_test), 
                             batch_size=batchSize)

#### STEP 3: Train the Model

We train using one of the three learning rates recommended in the BERT paper: *5e-5*, *3e-5*, or *2e-5*.
Alternatively, the ktrain Learning Rate Finder can be used to find a good learning rate by invoking `learner.lr_find()` and `learner.lr_plot()`, prior to training.

In [ ]:
# briefly simulate training to find good learning rate

#learner.lr_find(max_epochs=3)
#learner.lr_plot()

In [ ]:
learningRate = 2e-5
numEpoch = 3

# learner.fit_onecycle(learningRate, numEpoch)
learner.autofit(learningRate, numEpoch)



begin training using triangular learning rate policy with max lr of 2e-05...
Epoch 1/3
5500/5500 [==============================] - 1815s 328ms/step - loss: 0.0699 - accuracy: 0.9741 - val_loss: 0.0523 - val_accuracy: 0.9818
Epoch 2/3
5500/5500 [==============================] - 1798s 327ms/step - loss: 0.0320 - accuracy: 0.9894 - val_loss: 0.0550 - val_accuracy: 0.9823
Epoch 3/3
5500/5500 [==============================] - 1798s 327ms/step - loss: 0.0167 - accuracy: 0.9944 - val_loss: 0.0643 - val_accuracy: 0.9828


In [ ]:
learner.validate(val_data=(x_test, y_test))

              precision    recall  f1-score   support

           0       0.98      0.97      0.97      6139
           1       0.99      0.99      0.99     12719

    accuracy                           0.98     18858
   macro avg       0.98      0.98      0.98     18858
weighted avg       0.98      0.98      0.98     18858



array([[ 5951,   188],
       [  136, 12583]])

#### STEP 4: Making predictions

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
predictor.get_classes()

['Negative', 'Positive']

In [ ]:
print(predictor.predict(df_yelp.data.iloc[101]))

Positive


In [ ]:

# Save the entire model as a SavedModel.
#!mkdir -p saved_model
predictor.save(modelPath+'yelp_Bert_model')

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [ ]:
print(df_yelp.target.iloc[101])

Positive


In [ ]:
NumRecs = 50

target = df_yelp.target.iloc[0:NumRecs]
predicted = predictor.predict(df_yelp.data.iloc[0:NumRecs].tolist())
data = df_yelp.data.iloc[0:NumRecs]

results = pd.DataFrame(list(zip(target, predicted, data)), 
               columns =['target', 'predicted', 'data']) 

In [ ]:
results.head()

,target,predicted,data
0,Positive,Positive,"I'm SO glad I checked Yelp while walking in the North End trying to find a place to eat on a Saturday night. All the restaurants had long waits, and I didn't want to wait that long for just an average evening. \n\nComing to Bacco was anything but average. We only had to wait 30 minutes, we got a spacious table upstairs (anywhere would have been fine. Even the downstairs had all the windows open and a cool breeze was coming through this busy corner restaurant).\n\nThe food was awesome - I think anything on the menu would have been tasty, and I even saw a celebrity (Reshma who plays Divya from Royal Pains). She was so nice and we got our picture taken with her. \n\nEveryone from our server to the hostess to the manager who came over to see how our evening was going were great. And the patrons sitting next to us were even friendly. Overall, it was an amazing night to kick off our vacation in Boston."
1,Positive,Positive,"The sandwiches are out of this world, especially when you can see who designed them (top celebrity chefs). I've had the Schlesinger (sp?) and the meatloaf club, both were amazing. The servers do tend to get confused during the rush, at one point we had two servers waiting on us who would have brought us two separate meals if we chose to let them continue. Have yet to be able to sit outside, its always crowded. \nWill return for sure."
2,Positive,Positive,This place...will not quit! Awesome service. More than fair prices. You'd have to be out of your mind to give this place less than 5 stars. I'd give 'em six if I could. Great job to George and crew!
3,Positive,Positive,Vibrant interior. Good service. Amazing truffle grits.
4,Positive,Positive,"I have encountered many enjoyable experiences at both Face To Face Spa locations, Avery Ranch and Westlake. Although I have been perfectly satisfied with the Avery Ranch location, the Westlake location is about 30 minutes closer to my home, which is why my last two visits have been at Westlake.\n\nBesides being a lovely, comfortable facility, customer service at Westlake has been nothing less than top-notch. All of the ladies are extremely congenial and make you feel glad that you have chosen Face to Face for a relaxing, spa experience.\n\nI get a strong sense that my esthetician, Nicole, truly enjoys taking care of her clients. Nicole has approached each one of our sessions with enthusiasm, professionalism, and a genuine concern for my needs. Without a doubt, I will continue using Face to Face spa for expert exfoliation services!"


#### Load PreTrained Model 

In [ ]:
predictor = ktrain.load_predictor(modelPath+'yelp_Bert_model')

In [ ]:
#print(load_predictor.predict(df_yelp.data.iloc[101]))

### Step 5 : Predict Sentiment on News Articles

Read the news articles related to chicago and predict the sentiment for each of them

In [ ]:
valdt_json = pd.read_json(dataPath+"news_with_topic.json",orient='records',lines=True)
valdt_json.head()

,Unnamed: 0,date,language,title,text,text_clean,title_clean,rake_phrases,Top_Topic_Assigned,Perc_Contribution,Topic_Keywords
0,0,2021-07-23,english,chicago theater seating chart with seat numbers,Chicago Theatre Seating Chart With Seat Number...,Chicago Theatre Seating Chart With Seat Number...,chicago theater seating chart with seat numbers,mix wtmx chicago theatre seat numbers online c...,60.0,0.5188,"restaurant, food, order, pizza, menu, eat, ser..."
1,1,2021-07-24,english,harris theater chicago seating chart,Harris Theater Seatmap Chicago Opera Theater C...,Harris Theater Seatmap Chicago Opera Theater C...,harris theater chicago seating chart,harris theater seatmap chicago opera theater c...,46.0,0.2848,"chart, seating_chart, seat, guide, flight_deal..."
2,2,2020-12-15,english,There's a Big Tree Growing Through a Car in Il...,"Published: December 14, 2020 nanjan\nWe're not...","Published: December 14, 2020 nanjanWere not ex...",Theres a Big Tree Growing Through a Car in Ill...,illinois abandoned images troy shepard illinoi...,87.0,0.7097,"go, get, know, want, say, s, think, see, peopl..."
3,3,2020-03-05,english,Former Fox White House reporter Wendell Goler ...,"National News FILE - This May 4, 2010 file pho...","National News FILE This May 4, 2010 file phot...",Former Fox White House reporter Wendell Goler ...,2010 file photo shows fox news channels wendel...,6.0,0.4453,"say, vote, support, state, people, president, ..."
4,4,2020-12-30,english,I am looking forward to many things in 2021. I...,I am looking forward to many things in 2021. I...,I am looking forward to many things in 2021. I...,I am looking forward to many things in 2021. I...,"illinois basketball returning, weeks stories, ...",87.0,0.6493,"go, get, know, want, say, s, think, see, peopl..."


Drop articles with None/NA texts or title

In [ ]:
valdt_json.dropna(inplace=True)

In [ ]:
valdt_json.reset_index(inplace=True)

Save news articles after removing NA to retrieve later

In [ ]:
valdt_json.to_json(dataPath+"news_with_topic_rmNA.json",orient='records',lines=True)

In [ ]:
#valdt_json.text.iloc[6700:6702].tolist()

[None,
 'UPDATE: Illinois State Police investigating officer-involved shooting By KWQC Staff | Posted: Fri 10:23 PM, Jan 26, 2018 | Updated: Sat 3:34 PM, Jan 27, 2018 \nROCK FALLS, Ill. (KWQC) -- UPDATE: Illinois State Police are now investigating an officer involved shooting that happened in the 1300 block of Franklin Street in Rock Falls, Illinois. \nState Police Crime Scene Investigators processed the shooting scene and collected evidence from the involved officer and the person who was killed. They also spoke with witnesses early Saturday morning until 3 a.m. \nInvestigators also collected dash camera video from one Rock Falls squad car and two Whiteside County squad cars. \nPolice are not giving any details on regarding the shooting itself, nor are they releasing the name of the person who was killed. \nMultiple people who saw the incident take place tell TV-6 that some sort of police chase happened, and shots were fired. \nWhiteside County Coroner will perform an autopsy on the p

In [ ]:
valdt_json.iloc[6700:6702]

,Unnamed: 0,date,language,title,text,text_clean,title_clean,rake_phrases,Top_Topic_Assigned,Perc_Contribution,Topic_Keywords
6738,6738,2021-06-08,english,NBA news 2021: Luc Longley breaks silence on M...,"Australia’s three-time NBA champion, Luc Longl...","Australias threetime NBA champion, Luc Longley...",NBA news 2021: Luc Longley breaks silence on M...,hugely popular last dance documentary last yea...,77.0,0.2291,"team, season, player, bull, year, play, chicag..."
6739,6739,2021-06-08,english,"SDP vs CHC Dream11 Prediction | MLB, Major Lea...",SDP vs CHC Dream11 Prediction\nHere we come up...,SDP vs CHC Dream11 PredictionHere we come up w...,"SDP vs CHC Dream11 Prediction MLB, Major Leag...",san diego padres vs chicago cubs possible line...,94.0,0.2197,"say, family, child, woman, life, home, man, di..."


Predict the new articles based on pre-trained data from yelp

In [ ]:
#smpl_data = valdt_json[0:100]
predictor.batch_size = 128
predict_prob = predictor.predict(valdt_json.text.tolist(), return_proba=True)

Get the probabilities of the prediction and assign sentiment based on the probabilities

In [ ]:
import numpy as np

In [ ]:
pred_class = pd.DataFrame(predict_prob, columns=['neg_prob','pos_prob'])
pred_class['sentiment'] =  np.where(pred_class['neg_prob'] > pred_class['pos_prob'], 'Negative', 'Positive')

In [ ]:
pred_class.head(10)

,neg_prob,pos_prob,sentiment
0,0.008034,0.991966,Postive
1,0.055082,0.944918,Postive
2,0.039648,0.960352,Postive
3,0.008716,0.991284,Postive
4,0.000244,0.999756,Postive
5,0.056177,0.943824,Postive
6,0.945221,0.054779,Negative
7,0.463089,0.536911,Postive
8,0.811844,0.188156,Negative
9,0.908882,0.091118,Negative


In [ ]:
len(valdt_json),len(pred_class)

(194178, 194178)

In [ ]:
pred_class.groupby('sentiment').count()

,neg_prob,pos_prob
sentiment,,
Negative,84463,84463
Postive,109715,109715


In [ ]:
pred_class.to_csv(dataPath+'prediction_class.csv')

In [ ]:
final_json = pd.concat([valdt_json, pred_class], axis=1)

In [ ]:
len(final_json)

194178

In [ ]:
final_json.head(4)

,index,Unnamed: 0,date,language,title,text,text_clean,title_clean,rake_phrases,Top_Topic_Assigned,Perc_Contribution,Topic_Keywords,neg_prob,pos_prob,sentiment
0,0,0,2021-07-23,english,chicago theater seating chart with seat numbers,Chicago Theatre Seating Chart With Seat Number...,Chicago Theatre Seating Chart With Seat Number...,chicago theater seating chart with seat numbers,mix wtmx chicago theatre seat numbers online c...,60.0,0.5188,"restaurant, food, order, pizza, menu, eat, ser...",0.008034,0.991966,Postive
1,1,1,2021-07-24,english,harris theater chicago seating chart,Harris Theater Seatmap Chicago Opera Theater C...,Harris Theater Seatmap Chicago Opera Theater C...,harris theater chicago seating chart,harris theater seatmap chicago opera theater c...,46.0,0.2848,"chart, seating_chart, seat, guide, flight_deal...",0.055082,0.944918,Postive
2,2,2,2020-12-15,english,There's a Big Tree Growing Through a Car in Il...,"Published: December 14, 2020 nanjan\nWe're not...","Published: December 14, 2020 nanjanWere not ex...",Theres a Big Tree Growing Through a Car in Ill...,illinois abandoned images troy shepard illinoi...,87.0,0.7097,"go, get, know, want, say, s, think, see, peopl...",0.039648,0.960352,Postive
3,3,3,2020-03-05,english,Former Fox White House reporter Wendell Goler ...,"National News FILE - This May 4, 2010 file pho...","National News FILE This May 4, 2010 file phot...",Former Fox White House reporter Wendell Goler ...,2010 file photo shows fox news channels wendel...,6.0,0.4453,"say, vote, support, state, people, president, ...",0.008716,0.991284,Postive


Save the output results to drive for next steps

In [ ]:
final_json.to_json(dataPath+"news_with_Topics_Sentiments_proba.json",orient='records',lines=True)

Check the count of each sentiment

In [ ]:
final_json.groupby(['sentiment']).count()

,index,Unnamed: 0,date,language,title,text,text_clean,title_clean,rake_phrases,Top_Topic_Assigned,Perc_Contribution,Topic_Keywords,neg_prob,pos_prob
sentiment,,,,,,,,,,,,,,
Negative,84463,84463,84463,84463,84463,84463,84463,84463,84463,84463,84463,84463,84463,84463
Postive,109715,109715,109715,109715,109715,109715,109715,109715,109715,109715,109715,109715,109715,109715


In [ ]:
#valdt_json.groupby(['sentiment']).count()

,Unnamed: 0,date,language,title,text,text_clean,title_clean,rake_phrases,Top_Topic_Assigned,Perc_Contribution,Topic_Keywords
sentiment,,,,,,,,,,,
Negative,84463,84463,84463,84463,84463,84463,84463,84463,84463,84463,84463
Positive,109715,109715,109715,109715,109715,109715,109715,109715,109715,109715,109715


Explain the Classification using inbuilt predictor
This visualization is generated using a technique called LIME

In [ ]:
!pip install https://github.com/amaiya/eli5/archive/refs/heads/tfkeras_0_10_1.zip

  Using cached https://github.com/amaiya/eli5/archive/refs/heads/tfkeras_0_10_1.zip


In [ ]:
predictor.explain(valdt_json.text[0])